In [ ]:
import os
import torch as t
import torch.distributed as dist
import torch.multiprocessing as mp

In [ ]:
mp.set_start_method("spawn")

In [ ]:
def run(rank, size):
    print('I have been called')
    tensor = t.zeros(1)
    if rank == 0:
        tensor += 1
        # Send the tensor to process 1
        dist.send(tensor=tensor, dst=1)
    else:
        # Receive tensor from process 0
        dist.recv(tensor=tensor, src=0)
    print('Rank ', rank, ' has data ', tensor[0])
    
def init_process(rank, size, fn, backend='gloo'):
    """ Initialize the distributed environment. """
    os.environ['MASTER_ADDR'] = '127.0.0.1'
    os.environ['MASTER_PORT'] = '29501'
    dist.init_process_group(backend, rank=rank, world_size=size)
    fn(rank, size)

size = 2
processes = []
for rank in range(size):
    p = mp.Process(target=init_process, args=(rank, size, run))
    p.start()
    processes.append(p)

for p in processes:
    p.join()


I have been called


Process Process-2:
Traceback (most recent call last):
  File "/usr/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/usr/lib/python3.8/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-2-312ce34765fd>", line 18, in init_process
    fn(rank, size)
  File "<ipython-input-2-312ce34765fd>", line 3, in run
    tensor = torch.zeros(1)
NameError: name 'torch' is not defined


I have been called


Process Process-1:
Traceback (most recent call last):
  File "/usr/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/usr/lib/python3.8/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-2-312ce34765fd>", line 18, in init_process
    fn(rank, size)
  File "<ipython-input-2-312ce34765fd>", line 3, in run
    tensor = torch.zeros(1)
NameError: name 'torch' is not defined
Process Process-1:
Traceback (most recent call last):
  File "/usr/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/usr/lib/python3.8/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-2-312ce34765fd>", line 18, in init_process
    fn(rank, size)
  File "<ipython-input-2-312ce34765fd>", line 3, in run
    tensor = torch.zeros(1)
Process Process-1:
Traceback (most recent call last):
  File "/usr/lib/python3.8/multip

In [ ]:
def run(rank, size):
    # tensor = t.zeros(3,3)
    print('I have rank ', rank, ' and size is ', size)
    holder = t.zeros(3,3)
    group = dist.new_group(len(range(DEVICES)))
    if rank == 0:
        randt = t.randn(3,3)
        dist.broadcast(randt, 0, group)
    else: 
        dist.recv(tensor=holder, src=0)
        # does something
    # ? Why is this like this ?
    print('Rank ', rank, ' has data ', randt)

def init_processes(rank, size, fn, backend='Gloo'):
    """ Initialize the distributed environment. """
    os.environ['MASTER_ADDR'] = '127.0.0.1'
    os.environ['MASTER_PORT'] = '29501'
    dist.init_process_group(backend, rank=rank, world_size=size)
    print('I am initiating the process at:', rank, ' of ', size)
    fn(rank, size)
    
DEVICES=[0,1]
if __name__ == "__main__":
    size = len(DEVICES)
    processes = []
    # mp.set_start_method("spawn")
    for rank in range(size):
        p = mp.Process(target=init_processes, args=(rank, size, run))
        print("I am starting!", rank, size)
        p.start()
        processes.append(p)
    for p in processes:
        p.join()

Process Process-2:
Traceback (most recent call last):
  File "/usr/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/usr/lib/python3.8/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "<ipython-input-2-af2e81c3fb68>", line 21, in init_processes
    fn(rank, size)
  File "/usr/lib/python3.8/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-2-af2e81c3fb68>", line 5, in run
    group = dist.new_group(len(range(DEVICES)))
  File "<ipython-input-2-af2e81c3fb68>", line 21, in init_processes
    fn(rank, size)
TypeError: 'list' object cannot be interpreted as an integer
  File "<ipython-input-2-af2e81c3fb68>", line 5, in run
    group = dist.new_group(len(range(DEVICES)))
Process Process-2:
Traceback (most recen